In [280]:
import path_finder as pf
from Bio import SeqIO
import difflib
import json

In [318]:
def find_seq_path(seq, g):
    """Find the most likely path for a sequence through a given graph."""
    seq = seq.replace('-', '')
    path = []
    #  returns list of node ids and create list of those nodes
    head_node_ids = g.find_all_start_nodes()
    head_nodes = [g.nodes[node_id] for node_id in head_node_ids]
    # determine which node is the correct match for our sequence
    matches = find_match(seq, head_nodes)
    for match in matches:
        try:
            # remove the sequence of the match from the start of our sequence
            new_seq = seq[len(match.sequence):] if not match.sequence == '*' else seq
            # start recursive search for path through graph
            final_path = helper(new_seq, g, match.name, path + [match.name])
            return final_path
        except (AttributeError, TypeError) as err:
            if str(err) in ("'NoneType' object has no attribute 'name'", "'NoneType' object is not iterable"):
                continue
            else:
                raise(err)
            

def find_match(seq, nodes):
    """Find which node matches start of sequence and return which node matches"""
    matches = []
    
    for node in nodes:
        l = len(node.sequence)
        if node.sequence in ('*', seq[:l]):
            matches.append(node)
    # if we get to here then there was no exact match.
    # we need to return the node/s with * seq.
#     if not matches:
#         for node in nodes:
#             if node.sequence == '*':
#                 matches.append(node)
#     if len(matches) > 1: print(matches)
    return matches
    # if we get to here, theres a problem
#     print("Something went wrong. Shouldn't have gotten to here.")
#     print(seq)
#     print(nodes)
    
def helper(seq, g, start_from, paths_acc):
    """Helper function for path finder that recursively finds the correct path
    through the graph and will end once we hit the end of the sequence/graph."""
    # we have reached the end of the sequence or graph
#     print(paths_acc)
#     print(seq)
    if not start_from or not seq:
        return paths_acc
    nodes_to_try = [g.nodes[key] for key in g.nodes[start_from].out_edges]
    matches = find_match(seq, nodes_to_try)
    for match in matches:
        try:
            new_seq = seq[len(match.sequence):] if not match.sequence == '*' else seq
            return helper(new_seq, g, match.name, paths_acc + [match.name])
        except (AttributeError, TypeError) as err:
            if str(err) in ("'NoneType' object has no attribute 'name'", "'NoneType' object is not iterable"):
                continue
            else:
                raise(err)
    # raise this error as there are no matches, meaning we hit a dead end in the graph
    # and need to back-track. This error will cause the recursive helper function to back-track
    raise(TypeError("'NoneType' object is not iterable"))

def fasta_parser(filename):
    fasta = {}
    with open(filename, 'r') as f:
        contents = f.read()[1:].split('\n>')
        for section in contents:
            sample = section.split('\n')
            sample_id = sample[0]
            seq = ''.join(sample[1:]).strip()
            fasta[sample_id] = seq
    return fasta

def json_formatter(graph, fasta):
    paths = []
    for idx, (key, seq) in enumerate(fasta.items()):
        path_obj = {'id': idx, 'name': key, 'sequence': find_seq_path(seq, graph)}
        paths.append(path_obj)
    nodes = []
    for key, node in graph.nodes.items():
        node_obj = {'name': key, 'seq': node.sequence}
        nodes.append(node_obj)
    obj_to_write = {'nodes': nodes, 'paths': paths}
    return obj_to_write

## Test set
We will first test this on a little toy example.
The sequences for this test sample are:
```
>sample1
AGGAGCTTA
>sample2
ACGAGCTTA
>sample3
TCGAGCATA
```
And the GFA:
```
H       VN:Z:1.0        bn:Z:--linear --singlearr
S       0       *       RC:i:0
S       1       AG      RC:i:0
S       2       AC      RC:i:0
S       3       TC      RC:i:0
S       4       GAGC    RC:i:0
L       1       +       4       +       0M
L       2       +       4       +       0M
L       3       +       4       +       0M
L       4       +       5       +       0M
S       5       TTA     RC:i:0
L       4       +       6       +       0M
S       6       ATA     RC:i:0
S       7       *       RC:i:0
L       5       +       7       +       0M
L       6       +       7       +       0M
```

In [319]:
test_dir = '/Users/mbhall88/Dropbox/PhD/hiv_prg/data/testing/'
test_gfa_filename = test_dir + 'test.max_nest10.min_match3.gfa'
test_fasta_filename = test_dir + 'test.fasta'

Here we load in the GFA into the custom `Graph` data structure, along with the fasta file of the samples, and create the `json` object we need for the tubemaps. This structure requires nodes with their sequences and also path information.

In [323]:
test_graph = pf.Graph(from_gfa=test_gfa_filename)
test_fasta = fasta_parser(test_fasta_filename)
test_obj = json_formatter(test_graph, test_fasta)
test_obj

{'nodes': [{'name': '0', 'seq': '*'},
  {'name': '1', 'seq': 'AG'},
  {'name': '2', 'seq': 'AC'},
  {'name': '3', 'seq': 'TC'},
  {'name': '4', 'seq': 'GAGC'},
  {'name': '5', 'seq': 'TTA'},
  {'name': '6', 'seq': 'ATA'},
  {'name': '7', 'seq': '*'}],
 'paths': [{'id': 0, 'name': 'sample1', 'sequence': ['1', '4', '5']},
  {'id': 1, 'name': 'sample2', 'sequence': ['2', '4', '5']},
  {'id': 2, 'name': 'sample3', 'sequence': ['3', '4', '6']}]}

The paths for each sample above are correct!!
Let's write them to `json` and move on.

In [324]:
json_fname = test_fasta_filename[:-5] + 'json'
with open(json_fname, 'w') as fp:
    json.dump(test_obj, fp)

## Real dataset
We will now run the function on the HIV dataset and write it to file for reading into the sequence tubemap visualisation.

In [260]:
data_dir = '/Users/mbhall88/Dropbox/PhD/hiv_prg/data/'
gfa_filename = data_dir + 'five.max_nest10.min_match5.gfa'
hiv_fasta_filename = data_dir + 'HIV1_ALL_2016_pol_DNA_5_samples.fasta'

In [325]:
hiv_graph = pf.Graph(from_gfa=gfa_filename)
hiv_fasta = fasta_parser(hiv_fasta_filename)
hiv_obj = json_formatter(hiv_graph, hiv_fasta)
# quick check to ensure paths not empty
for d in hiv_obj['paths']:
    print(len(d['sequence']))

499
513
527
533
537


Now that we have our paths, let's write them to `json` along with the details of all the nodes.

In [326]:
json_fname = hiv_fasta_filename[:-5] + 'json'
with open(json_fname, 'w') as fp:
    json.dump(hiv_obj, fp)